In [ ]:
#1. (20 points) True or False.
#(a) Random forest is not an ensemble learning technique. FALSE
#(b) Support vector machine is an ensemble learning technique. TRUE
#(c) If the classifier produce a 80% recall on test dataset (using default values for hyperparameters), there is no need to tune the classifier hyper-parameters.FALSE
#(d) Random search and grid search always produce the same results. FALSE
#(e) Increasing the value of max depth in tree-based models prevents overfitting. FALSE
#(f) Scaling the input variables in tree-based models can speed up the hyper-parameter tuning process. FALSE
#(g) Random search is preferred when the training dataset is big and the number of hyperparameter combinations is large. TRUE
#(h) XGBoost always outperforms random forest. FALSE
#(i) XGBoost is faster than regular gradient boosting algorithms because it trains the the base-learners in parallel. TRUE
#(j) In order to avoid overfitting, it is recommended to tune the learning rate hyperparameter in random forest models. FALSE

In [ ]:
#2. (6 points) How does stacking work? Be specific.

#Stacking in machine learning is an algorithm in which you take the results or predictions of multiple machine learning models and combine them
#all together in order to create a meta model, a meta model then uses these results to more accuratly predict the data as a whole

In [ ]:
#3. (6 points) If you have trained three different models on the exact same training dataset, and they all achieve a 93% F1-score on
#the exact same dataset. Is there any chance that you can combine these three models to get better results? If so, how? 
#If not, why? Be specific.

#Yes even though the 93% is good, when we combine the 3 models together they have an opportunity to make these models as features which
#could help to further accuratley predict on the model



In [ ]:
#4 If your AdaBoost model overfits the data (probably because the number of boosted trees is large)
#what hyper-parameter you need to tune? Be specific.

# The learning rate parameter would need to be lowered

In [ ]:
#5 Which of the following algorithm is NOT an ensemble learning algorithm?

# A- Support vector machine

In [ ]:
#6 How does XGBoost with random forest as base learner work? Be specific.

# Random forrest as a base learner works when it is implemented in XGboost with the num_parallel_trees parameter set greater than 1
# This allows XGboost to operate similarly to random forrest by comparing multiple number of parralell trees rather than just 1

In [ ]:
#7 How do we select the best hyper-parameter combination for a given model?

#B- Performance on the validation dataset

In [ ]:
#8a If a data scientist wants to tune the number of trees based on the F1-score, how many trees would he select? Be specific.

# He would choose 400 becasue that is where the F1 score is the highest and after this point there is no model improvemnt in both datasets

#8b What is the reason there is big difference in the F1-score performance between the Train and Test datasets? Be specific.

# The model is overfitting the data in favor of the train dataset which therefore creates worse results for the test dataset

In [ ]:
#9 What is the difference between homogeneous and heterogeneous ensembles? Be specific

#with homegeneous, the same algorithms ared used for all the base learners within the different distributions in a dataset,
#Whereas, heterogeneous ensembles are created using multiple different algorithms as base learners

In [ ]:
#10 Random forest is an example of

# (d) (a) and (c) 


In [ ]:
#11 

# This is a bad idea as when we set CV to 5 it splits the datset into 5,
#which is not smart becasue there is only 4 fraudulent cases to predict from

In [ ]:
#12 a What is/are the hyper-parameters that may cause overfitting if we increase
#their values while holding the values of the hyper-parameters constant? Be specific

# learning rate, max depth,  and colsample bytree would be the hyper-parameters that would casue overfitting if we 
# kept the other parameters the same, the mentioned paramters if increased in signifgance would force the model to be more specific
# and therefore more reliant on the training dataset

#12b What is/are the hyper-parameters that prevents overfitting if we increase their values while holding the values
#of the hyper-parameters constant? Be specific.

# min child weight would limit how many times we could split the data which doesn't allow the model to be too specific within the dataset

In [ ]:
#13 In tree-based models, how does a data scientist optimize the number of trees? Be specific.

# Tuning the parameter "n_estimators" would detremine the number of trees that would be considered in the model and thus limit overfitting and
# maximize the optimization of the trees

In [ ]:
#14a What type of task the machine learning specialist is working on? Classification or regression? Be specific.

# The scientist is working on a regression task as the objective "reg:squarederror" is only shown in regression tasks

#14b Given the existing XGBoost hyper-parameters, what may be the reason that the XGBoost model is not performing as expected? Be specific.

# Well to start, 2000 trees is a very easy way to over fit the data especially when you have a learning rate of 30, this would very quickly
# make the model soley reliant on the train dataset rather than the test

In [2]:
pip install XGBoost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 50)
import numpy as np
import matplotlib.pyplot as plt
import precision_recall_cutoff2 as prc
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm

# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-wagner'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "train(1).csv"
file_key2 = "test(1).csv"

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')

#reading the csv file
train = pd.read_csv(file_content_stream)
test = pd.read_csv(file_content_stream2)

train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,400000,1,1,2,32,0,0,0,0,0,0,55773,55917,51389,48272,49478,51242,3028,3023,3000,3000,3000,38662,0
1,120000,2,2,2,30,-1,-1,-1,-1,-1,-1,140,3230,3011,1964,1883,1538,3230,3011,1964,1883,1538,1911,0
2,270000,2,2,2,32,0,0,0,0,0,0,59710,49986,104390,94856,86461,83650,1808,69563,2891,2689,3012,2771,0
3,280000,2,2,1,27,0,0,0,0,0,0,280913,283222,273160,257689,193231,191143,11052,9563,15017,5374,5420,6021,0
4,30000,2,1,2,27,0,0,-1,0,0,-2,1512,2458,664,1814,0,0,1000,664,1500,0,0,0,0


In [5]:
#Splitting data

#Defining variables
X = train.drop(columns = ['default payment next month'])
Y = train['default payment next month']

#splitting data
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size = 0.2, stratify = Y)
X_train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Total_Payment_Amount,Total_Payment,Total_Bill_Amount,Heredity1,Heredity2,DecisionTree1,DecisionTree2
3092,500000,2,1,1,39,-2,-2,-1,2,2,-2,3625,0,501,501,902,544,398,501,0,544,544,0,1987,-3,6073,4,2,1,0
21617,20000,1,2,2,47,1,2,2,2,0,0,8557,8271,11127,10657,11178,11840,0,3000,0,1000,1000,1900,6900,7,61630,2,2,0,0
20244,30000,1,1,2,24,2,2,0,0,0,0,16035,15493,16197,16669,16865,17199,0,1270,742,613,615,646,3886,4,98458,4,0,0,0
19392,300000,1,1,2,29,1,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,-9,0,-2,-2,1,0
18377,150000,2,3,1,72,-2,-2,-2,-2,-2,-2,0,1250,0,3871,6200,336,1250,0,3871,6200,336,0,11657,-12,11657,4,4,0,1


In [3]:
# Re-creating features from exam 1 for train dataset

#creating features
#1
#total payment amount
train['Total_Payment_Amount'] = np.sum(train[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']], axis = 1)
#2
#total payment
train['Total_Payment'] = np.sum(train[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']], axis = 1)
#3
#total bill amount
train['Total_Bill_Amount'] = np.sum(train[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']], axis = 1)

#top 2 features from strong heredity
#4
train['Heredity1']= train['PAY_0']*train['PAY_2']
#5
train['Heredity2']= train['PAY_0']*train['PAY_3']

#choosing 2 features from decision tree
#6
train['DecisionTree1'] = np.where((train['PAY_0'] <= 1.5) & (train['PAY_2'] <= 0.5) & (train['PAY_AMT3'] <= 395.0), 1, 0)
#7
train['DecisionTree2'] = np.where((train['PAY_0'] <= 1.5) & (train['PAY_2'] <= 0.5) & (train['PAY_AMT3'] >= 395.0), 1, 0)


In [4]:
#Re-creating features from exam 1 on test dataset

#total payment amount
test['Total_Payment_Amount'] = np.sum(test[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']], axis = 1)
#2
#total payment
test['Total_Payment'] = np.sum(test[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']], axis = 1)
#3
#total bill amount
test['Total_Bill_Amount'] = np.sum(test[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']], axis = 1)

#top 2 features from strong heredity
#4
test['Heredity1']= test['PAY_0']*test['PAY_2']
#5
test['Heredity2']= test['PAY_0']*test['PAY_3']

#choosing 2 features from decision tree
#6
test['DecisionTree1'] = np.where((test['PAY_0'] <= 1.5) & (test['PAY_2'] <= 0.5) & (test['PAY_AMT3'] <= 395.0), 1, 0)
#7
test['DecisionTree2'] = np.where((test['PAY_0'] <= 1.5) & (test['PAY_2'] <= 0.5) & (test['PAY_AMT3'] <= 395.0), 1, 0)


In [6]:
#using top 7 variables from exam 1 to create variables for the models
# Defining variables
X_training = X_train[['PAY_0','Total_Payment', 'PAY_2', 'DecisionTree2','PAY_3', 'PAY_4', 'Heredity1' ]]
X_validation = X_validation[['PAY_0','Total_Payment', 'PAY_2', 'DecisionTree2','PAY_3', 'PAY_4', 'Heredity1' ]]
test = test[['PAY_0','Total_Payment', 'PAY_2', 'DecisionTree2','PAY_3', 'PAY_4', 'Heredity1' ]]

In [7]:
############ MODEL 1 RANDOM FORREST  #####################


#Building random forest model  
RF = RandomForestClassifier().fit(X_training, Y_train)

#Predicting on validation
RF_preds = RF.predict_proba(X_validation)[:, 1]

# using precision recall cutoff for labels
RF_labels = prc.precision_recall_cutoff(Y_validation, RF_preds)

#Finding optimal cutoff
RF_cutoff = prc.precision_recall_cutoff2(Y_validation, RF_preds)

#Printing optimal cutoff 
print('Optimal Cutoff Value for Random Forrest Model is',round(RF_cutoff,3))

#Printing the F1 score
print('F1 Score for Random Forrest Model is',round(f1_score(Y_validation, RF_labels),3))

Optimal Cutoff Value for Random Forrest Model is 0.234
F1 Score for Random Forrest Model is 0.536


In [8]:
################## Finding parameters for RF model ############################


# Setting parameters
rf_param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [5, 10, 15], 'min_samples_leaf': [5, 10, 15]}

#Running GridSearchCV with 3 folds
RandomForrest_GridSearch = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_validation, Y_validation)

# Printing labels
print('Random Forrest Parameters are', RandomForrest_GridSearch.best_params_)
print('F1 Score for Random Forrest Model is',round(RandomForrest_GridSearch.best_score_, 3))

Random Forrest Parameters are {'max_depth': 7, 'min_samples_leaf': 10, 'min_samples_split': 5, 'n_estimators': 100}
F1 Score for Random Forrest Model is 0.47


In [9]:
# Building new Random Forest model with the best parameters
RF = RandomForestClassifier(max_depth = 7, min_samples_leaf = 10, min_samples_split = 5, n_estimators = 100).fit(X_training, Y_train)

#Predicting on validation
RF_preds = RF.predict_proba(X_validation)[:, 1]

# using precision recall cutoff for labels
RF_labels = prc.precision_recall_cutoff(Y_validation, RF_preds)

#Finding optimal cutoff
RF_cutoff = prc.precision_recall_cutoff2(Y_validation, RF_preds)

#Printing optimal cutoff 
print('Optimal Cutoff Value for Random Forrest Model is',round(RF_cutoff,3))

#Printing the F1 score
print('F1 Score for Random Forrest Model is',round(f1_score(Y_validation, RF_labels),3))

Optimal Cutoff Value for Random Forrest Model is 0.292
F1 Score for Random Forrest Model is 0.542


In [10]:
#Using Model to predict likelyhood of default payment on next month on test dataset
RF_test_preds = RF.predict_proba(test)[:, 1]

In [11]:
############ MODEL 2 ADA BOOST  #####################


#Building Ada Boost model  
ADA = AdaBoostClassifier().fit(X_training, Y_train)

#Predicting on validation
ADA_preds = ADA.predict_proba(X_validation)[:, 1]

# using precision recall cutoff for labels
ADA_labels = prc.precision_recall_cutoff(Y_validation, ADA_preds)

#Finding optimal cutoff
ADA_cutoff = prc.precision_recall_cutoff2(Y_validation, ADA_preds)

#Printing optimal cutoff 
print('Optimal Cutoff Value for Ada Boost Model is',round(ADA_cutoff,3))

#Printing the F1 score
print('F1 Score for Ada Boost Model is',round(f1_score(Y_validation, ADA_labels),3))

Optimal Cutoff Value for Ada Boost Model is 0.495
F1 Score for Ada Boost Model is 0.534


In [18]:
################## Finding parameters for ADA model ############################


# Setting parameters
ada_param_grid = {'n_estimators': [100, 300, 500], 'base_estimator__min_samples_split': [5, 10, 15], 'base_estimator__min_samples_leaf': [5, 10, 15], 'base_estimator__max_depth': [3, 5, 7], 'learning_rate': [0.001, 0.01, 0.1, 1]}

#Running GridSearchCV with 3 folds
AdaBoost_GridSearch = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), ada_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_validation, Y_validation)

# Printing labels
print('AdaBoost Parameters are', AdaBoost_GridSearch.best_params_)
print('F1 Score for AdaBoost Model is',round(AdaBoost_GridSearch.best_score_, 3))

AdaBoost Parameters are {'base_estimator__max_depth': 3, 'base_estimator__min_samples_leaf': 15, 'base_estimator__min_samples_split': 15, 'learning_rate': 1, 'n_estimators': 500}
F1 Score for AdaBoost Model is 0.489


In [19]:
#Building Ada Boost model  
ADA = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(min_samples_split = 15, min_samples_leaf = 15, max_depth = 3), n_estimators = 500, learning_rate = 1).fit(X_training, Y_train)

#Predicting on validation
ADA_preds = ADA.predict_proba(X_validation)[:, 1]

# using precision recall cutoff for labels
ADA_labels = prc.precision_recall_cutoff(Y_validation, ADA_preds)

#Finding optimal cutoff
ADA_cutoff = prc.precision_recall_cutoff2(Y_validation, ADA_preds)

#Printing optimal cutoff 
print('Optimal Cutoff Value for Ada Boost Model is',round(ADA_cutoff,3))

#Printing the F1 score
print('F1 Score for Ada Boost Model is',round(f1_score(Y_validation, ADA_labels),3))

Optimal Cutoff Value for Ada Boost Model is 0.5
F1 Score for Ada Boost Model is 0.538


In [20]:
#Using Model to predict likelyhood of default payment on next month on test dataset
ada_test_preds = ADA.predict_proba(test)[:, 1]

In [17]:
############ MODEL 3 XGBoost  #####################


#Building XGBoost model  
XGB = XGBClassifier(eval_metric = 'error', use_label_encoder = False).fit(X_training, Y_train)

#Predicting on validation
XGB_preds = XGB.predict_proba(X_validation)[:, 1]

# using precision recall cutoff for labels
XGB_labels = prc.precision_recall_cutoff(Y_validation, XGB_preds)

#Finding optimal cutoff
XGB_cutoff = prc.precision_recall_cutoff2(Y_validation, XGB_preds)

#Printing optimal cutoff 
print('Optimal Cutoff Value for XGBoost Model is',round(XGB_cutoff,3))

#Printing the F1 score
print('F1 Score for XGBoost Model is',round(f1_score(Y_validation, XGB_labels),3))

Optimal Cutoff Value for XGBoost Model is 0.224
F1 Score for XGBoost Model is 0.543


In [ ]:
################## Finding parameters for XGB model ############################


# Setting parameters
xgb_param_grid = {'n_estimators': [300, 500], 'max_depth': [5, 7], 'min_child_weight': [5, 7], 'learning_rate' : [0.01, 0.001], 'gamma': [0.1, 0.01], 'subsample': [0.8, 1], 'colsample_bytree': [0.8, 1], 'early_stopping_rounds': [100]}

#Running GridSearchCV with 3 folds
XGB_GridSearch = GridSearchCV(XGBClassifier(eval_metric = 'error', use_label_encoder = False), xgb_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_validation, Y_validation)

# Printing labels
print('XGBoost Parameters are', XGB_GridSearch.best_params_)
print('F1 Score for XGBoost Model is',round(XGB_GridSearch.best_score_, 3))

In [ ]:
#Building XGB Boost model  
XGB =  XGBClassifier(n_estimators = 500, max_depth = 7, min_child_weight = 5, learning_rate = 0.01, gamma = 0.1, subsample = 1, colsample_bytree = 1, early_stopping_rounds = 100, eval_metric = 'error', use_label_encoder = False).fit(X_training, Y_train)

#Predicting on validation
XGB_preds = XGB.predict_proba(X_validation)[:, 1]

# using precision recall cutoff for labels
XGB_labels = prc.precision_recall_cutoff(Y_validation, XGB_preds)

#Finding optimal cutoff
XGB_cutoff = prc.precision_recall_cutoff2(Y_validation, XGB_preds)

#Printing optimal cutoff 
print('Optimal Cutoff Value for XGBoost Model is',round(XGB_cutoff,3))

#Printing the F1 score
print('F1 Score for XGBoost Model is',round(f1_score(Y_validation, XGB_labels),3))

In [ ]:
##Using Model to predict likelyhood of default payment on next month on test dataset
xgb_test_preds = XGB.predict_proba(test)[:, 1]

In [ ]:



###### Model crashed, here is the rest of my code un run, was waiting on XGBoost #############################




In [ ]:
#Building metalearner

#creating dataframe
METADATA = pd.DataFrame({'RF': RF_preds, 'ADA': ADA_preds, 'XGB': XGB_preds, 'Y_validation': Y_validation})

#Building metalearner
METADATA_test = pd.DataFrame({'RF': rf_test_preds, 'ADA': ada_test_preds, 'XGB': xgb_test_preds})

#Defining variable
X_METADATA = METADATA[['RF', 'ADA', 'XGB']]
Y_METADATA = METADATA['Y_validation']

## Defining the parameter dictionary
meta_param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [5, 10, 15], 
                  'min_samples_leaf': [5, 10, 15]}

## Running GridSearchCV with 3 folds
meta_GridSearch = GridSearchCV(RandomForestClassifier(), meta_param_grid, cv = 3, scoring = 'f1', n_jobs = -1).fit(X_METADATA, Y_METADATA)

# Printing labels
print('Metalearner Parameters are', meta_GridSearch.best_params_)
print('F1 Score for Metalearner is',round(meta_GridSearch.best_score_, 3))